In [2]:
import julia 
from julia import Julia
jl = Julia(compiled_modules=False)  # Initialize Julia interpreter
Julia(runtime="/home/jxxiong/julia-1.9.2/bin/julia")
from julia import Base
# Now you can call functions from Julia's Base module
from julia import Main

In [2]:
import numpy as np
import gurobipy as gp

from gurobipy import Model, GRB, QuadExpr, LinExpr
from scipy.sparse import csc_matrix
import pickle

In [3]:
case_dict = {200: "pglib_opf_case200_activ.m",
             2000: "pglib_opf_case2000_goc.m",
             10000: "pglib_opf_case10000_goc.m"}

In [1]:
nbus = 2000
case_name = case_dict[nbus]
Main.case_name = case_name
Main.nbus = nbus

n_samples = 10000
perturb_rate = 0.3

NameError: name 'case_dcit' is not defined

In [5]:
Main.eval("""
using PowerModels
using JuMP
using Ipopt

file_path = "/home/jxxiong/A-xjx/DC3/datasets/dcopf/";
network_data_orig = PowerModels.parse_file(file_path*case_name);
optimizer = optimizer_with_attributes(Ipopt.Optimizer, "print_level"=>0);
""")

[info | PowerModels]: removing 1 cost terms from generator 41: [1872.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 65: [3000.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 335: [3000.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 326: [2662.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 299: [3000.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 168: [3000.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 159: [3000.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 332: [3000.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 190: [2713.0, -6.071]
[info | PowerModels]: removing 1 cost terms from generator 223: [2852.0, -18.4]
[info | PowerModels]: removing 1 cost terms from generator 88: [3000.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 297: [2534.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 26:

<PyCall.jlwrap MathOptInterface.OptimizerWithAttributes(Ipopt.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawOptimizerAttribute("print_level") => 0])>

In [6]:
def get_constraints(m):
    ini_ineq_matrix = []
    ini_ineq_rhs = []
    ini_eq_matrix = []
    ini_eq_rhs = []
    a_matrix = m.getA().toarray()

    i = 0
    for constr in m.getConstrs():
        if constr.sense == '<':
            ini_ineq_matrix.append(a_matrix[i,:])
            ini_ineq_rhs.append(constr.RHS)
            i += 1
        elif constr.sense == '>':
            ini_ineq_matrix.append(-a_matrix[i,:])
            ini_ineq_rhs.append(-constr.RHS)
            i += 1
        elif constr.sense == '=':
            ini_eq_matrix.append(a_matrix[i,:])
            ini_eq_rhs.append(constr.RHS)
            i += 1

    ini_ineq_matrix = np.array(ini_ineq_matrix)
    ini_ineq_rhs = np.array(ini_ineq_rhs)
    ini_eq_matrix = np.array(ini_eq_matrix)
    ini_eq_rhs = np.array(ini_eq_rhs)

    return ini_ineq_matrix, ini_ineq_rhs, ini_eq_matrix, ini_eq_rhs

def get_objective(m):
    ini_q_matrix = m.getQ().toarray()
    ini_p_vec = []
    variables = m.getVars()
    for var in variables:
        ini_p_vec.append(var.obj)
    ini_p_vec = np.array(ini_p_vec)

    return ini_q_matrix, ini_p_vec

def get_bound(m):
    variables = m.getVars()
    ini_lb = []
    ini_ub = []
    for var in variables:
        ini_lb.append(var.lb)
        ini_ub.append(var.ub)
    ini_lb = np.array(ini_lb)
    ini_ub = np.array(ini_ub)

    return ini_lb, ini_ub

def perturb_eq_rhs(ini_eq_rhs, num_samples, perturb_rate=perturb_rate):
    eq_rhs_list = []
    for _ in range(num_samples):
        eq_rhs = ini_eq_rhs.copy()
        nonzero_indices = np.nonzero(ini_eq_rhs)
        nonzero_values = ini_eq_rhs[nonzero_indices]
        perturbed_values = nonzero_values * (1 + np.random.uniform(-perturb_rate, perturb_rate, size=nonzero_values.shape))
        eq_rhs[nonzero_indices] = perturbed_values
        eq_rhs_list.append(eq_rhs)
    return np.array(eq_rhs_list)

## Output the .lp file

In [7]:
Main.eval("""
pm = instantiate_model(network_data_orig, DCPPowerModel, PowerModels.build_opf);
JuMP.write_to_file(pm.model, "dcopf$(nbus)_model.lp")
""")

In [8]:
G, h, A, x = get_constraints(m)
Q, p = get_objective(m)
Lb, Ub = get_bound(m)
X = perturb_eq_rhs(x, n_samples)

num_var = p.shape[0]
num_eq = A.shape[0]
num_ineq = G.shape[0]

print(f"number of variables: {num_var}, number of equality constraints: {num_eq}, number of inequality constraints: {num_ineq}")

Read LP format model from file /home/jxxiong/A-xjx/DC3/datasets/dcopf/dcopf2000_model.lp
Reading time = 0.02 seconds
obj: 11246 rows, 5872 columns, 29628 nonzeros


In [9]:
data = {'Q':Q, 'p':p, 'A':A, 'X':X, 'G':G, 'h':h, 'Lb':Lb, 'Ub':Ub, 'Y':[]}
with open(f"dcopf{nbus}_data1", 'wb') as f:
    pickle.dump(data, f)

(5612, 5872) (5612,) (5634, 5872) (5872, 5872) (5872,) (5872,) (5872,)


## Load Data

In [ ]:
with open(f"dcopf{nbus}_data", 'rb') as f:
    dataset = pickle.load(f)

# Other

In [ ]:
nonzero_indices = np.nonzero(Q)
print(nonzero_indices)


(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]), array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]))


In [ ]:
np.nonzero(p)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]),)

In [ ]:
lb_index = np.where(Lb != -np.inf)[0]
ub_index = np.where(Ub != np.inf)[0]

In [ ]:
np.where(Lb == Ub)[0]

array([240, 349, 382, 403, 434, 483])

In [ ]:
intersection = np.intersect1d(lb_index, ub_index)
print(intersection)

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32 233 234 235 236
 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254
 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272
 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290
 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306 307 308
 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324 325 326
 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 342 343 344
 345 346 347 348 349 350 351 352 353 354 355 356 357 358 359 360 361 362
 363 364 365 366 367 368 369 370 371 372 373 374 375 376 377 378 379 380
 381 382 383 384 385 386 387 388 389 390 391 392 393 394 395 396 397 398
 399 400 401 402 403 404 405 406 407 408 409 410 411 412 413 414 415 416
 417 418 419 420 421 422 423 424 425 426 427 428 429 430 431 432 433 434
 435 436 437 438 439 440 441 442 443 444 445 446 44

In [ ]:
np.mean(data['objective'])

13405.669006776963

In [ ]:
np.mean(data['solve_time'])

0.00903625726699829

In [ ]:
len(data['objective'])

100